In [1]:
#Test notebook for tracking rigid body  marker consisting of multiple fiducials within a CT scan volume

#3D data processing
import open3d as o3d
import ct_tracking_library # our helper functions
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, fixed

#our processing code
from ct_tracking_library.ct_motor_functions import *
from ct_tracking_library.ct_tracking_functions import *
from ct_tracking_library.ct_processing_functions import *
from ct_tracking_library.ct_display_functions import *
from ct_tracking_library.ct_object_classes import *
from ct_tracking_library.ct_fk_functions import *

#used for GUI
%matplotlib notebook

#find transform
import datetime
import time
import pickle
from spatialmath import *
from roboticstoolbox import ET as E
import transforms3d as t3d
import itertools
from spatialmath.base import trnorm

In [2]:
def DICOM_series_dropdown_callback(series_to_load, series_dictionary):
    """
    This function is a helper function that helps the drop dwon list for DICOM file in folder:

    Args:
        series_to_load(list): list of the DICOM serial number in the folder.
        series_dictionary(dict): dict that holds the dicm files.

    """
    global selected_series
               # Print some information about the series from the meta-data dictionary
               # DICOM standard part 6, Data Dictionary: http://medical.nema.org/medical/dicom/current/output/pdf/part06.pdf
    file_reader.SetFileName(series_dictionary[series_to_load][0])
    file_reader.ReadImageInformation()
    tags_to_print = {'0010|0010': 'Patient name: ', 
                     '0008|0060' : 'Modality: ',
                     '0008|0021' : 'Series date: ',
                     '0008|0080' : 'Institution name: ',
                     '0008|1050' : 'Performing physician\'s name: '}
    for tag in tags_to_print:
        try:
            print(tags_to_print[tag] + file_reader.GetMetaData(tag))
        except: # Ignore if the tag isn't in the dictionary
            pass
    selected_series = series_to_load

In [13]:
def get_marker(selected_series,time,v = False, debug = False):
    """
    This function finds the marker from the user sleceted dicom file:

    Args:
        selected_series(string): the serial key for the slected DICOM.
        time(Time): time intial when the function is called.
        v(bool): to show visualization for the marker
        debug(bool): wheather or not the user want to see debug info.

    Returns:
        M_list[0](Marker): the moveing marker object finded
        M_list[1](Marker): the fixed marker object finded
        v_mesh(Open3d mesh): the mesh used for finding the marker in form of open3d

    """
    M_list = []
    data_directory = "./test_data/DICOM"
    file_reader = sitk.ImageFileReader()
    series_file_names = {}
    series_IDs = list(sitk.ImageSeriesReader_GetGDCMSeriesIDs(data_directory))
    if series_IDs:
        for series in series_IDs:
            series_file_names[series] = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, series)
    for S in range(2):
        # pick marker
        if S == 0:
            target_marker_name = 'moving_marker_crane'
            marker = np.load('./test_data/marker1.npy')
            #marker = np.array([[-10,-5,0],[-10,5,0],[0,-5,0],[10,0,0]])
            #R = np.eye(3)
            #r_y = np.eye(3)
            #r_y[0][0] = -1
            #r_y[2][2] = -1
            #marker = (r_y@marker.T).T
            crop_z = None
            color = [0.8, 0.2, 0.8]
        else:
            target_marker_name = 'fixed_marker_crane'
            marker = np.load('./test_data/marker2.npy')
            #marker = marker - marker.mean(axis=0) - np.array([2,0,0])
            #R = np.eye(3)
            #r_z = np.eye(3)
            #r_z[0][0] = -1
            #r_z[1][1] = -1
            #marker = (r_z@marker.T).T
            crop_z = None
            color = [0.2, 0.2, 0.8]
        #convert scan to mesh for finding markers via thresholding
        scan_file = series_file_names[selected_series]
        convert_scan_to_mha(scan_file, crop_z = crop_z)
        convert_mha_to_mesh(output_mesh_file = 'temp_mesh.obj')

        #find candidate fiducials within the scanner returning clustered centroids that could be rigid body markers
        _, marker_centroid_coordinates, pcd_selected, marker_mesh, _, good_centroid_clusters = find_candidate_centroids(marker = marker, debug=debug)
        if debug:
            print('marker centroid coordinates from design: {}'.format(marker))
            print('found centroid coordinates: {}'.format(marker_centroid_coordinates))

        #display candidate fiducials
        if v:
            o3d.visualization.draw_geometries([pcd_selected, marker_mesh])

        #find best transform and marker from candidate fiducials
        tracked_R, tracked_t, permuted_centroids, min_error = find_best_transform_from_candidate_marker_clusters(marker, good_centroid_clusters)
        #get rid of column of ones to make 3xN
        permuted_centroids = permuted_centroids[:3,:].T

        #transform marker geometry coordinates using found transform to tracked image coordinates
        transformed_marker = (tracked_R @ marker.T + tracked_t).T
        error = permuted_centroids-transformed_marker
        
        #tracking error from final transform
        np.set_printoptions(2)
        print("Error (mm): {}".format(error))
        print("Error norm (mm): {}".format(np.linalg.norm(error)))
        if v:
            #create o3d visualizations
            marker_3d_tracked = create_marker_visualization(fiducial_coordinates=permuted_centroids, color=[0.8, 0.2, 0.2])
            marker_3d_base = create_marker_visualization(marker, [0.2, 0.8, 0.2])
            marker_3d_base.append(create_coordinate_frame_visualization())
            marker_3d_transformed = create_marker_visualization(marker, [0.2, 0.2, 0.8], tracked_R, tracked_t)
            marker_3d_transformed.append(create_coordinate_frame_visualization(tracked_R, tracked_t))
            #visualize tracked markers on top of mesh used for tracking
            visualization_list2 = marker_3d_tracked+marker_3d_base+marker_3d_transformed + [marker_mesh]
            visualization_list1 = copy.deepcopy(visualization_list2)
            o3d.visualization.draw_geometries(visualization_list2)
        if S == 0:
            v_mesh= marker_mesh
        #append marker
        print("T find: ",tracked_t)
        marker_3d = create_marker_visualization(marker, color, tracked_R, tracked_t)
        marker_3d.append(create_coordinate_frame_visualization(tracked_R, tracked_t))
        M_list.append(Marker(name=target_marker_name,d_key=selected_series,time =time,geometry=marker,T=SE3(t2T(tracked_R, tracked_t)),error=np.linalg.norm(error, axis=1)))
        
    return M_list[0],M_list[1],v_mesh

In [4]:
def calc_DLS_controller_step(J, e, alpha = 1.0, lambd = 0.01):
    '''
    calculates new position step using dls jacobian
    inputs: 
            J: [6xN] numpy array where N is the number of robot joints being controlled
            e: [6x1] numpy array for position and orientation errors stacked
            alpha: gain constant for position step
            lambd: damping constant for inverse
    return: 
        joint position setpoint delta
    '''
    J_inv = np.linalg.inv(J.T@J + lambd * np.eye(J.shape[1])) @ J.T
    return alpha * J_inv @ e

In [5]:
def close_loop(m1,m2,fk,r_error=None,p_error=None,Dof=True):
    if Dof:
        Tfk = SE3(m2.T).inv()*fk
        T2 = copy.deepcopy(Tfk)
        q_current = (SE3(m2.T).inv()*SE3(m1.T)).t[2]
        J = np.array([[0,0,1,0,0,0]]).T
        e = np.hstack((p_error.T[0],r_error))
        delta_q = calc_DLS_controller_step(J, e)
        print('Joint step calculated: {}'.format(delta_q))
        q_new = q_current + delta_q
        T2.t = [Tfk.t[0],Tfk.t[1],q_new]
        return T2,delta_q
    else:
        print("Crane bot")
        martix = np.array(([0,.04,-1,-0.21],[1,0.2,0,-1.87],[.02,-1,-.04,.98],[0,0,0,1]))
        rw_T_rb = SE3(martix)
        Base = SE3(m2)
        Tracked = rw_T_rb*Base.inv()*SE3(m1)
        Target = rw_T_rb*Base.inv()*SE3(fk)
        p_error = calc_p_error(Target.t,Tracked.t)
        r_error = t3d.euler.axangle2euler(calc_needle_ori_error(Target.R,Tracked.R))
        print("P: ")
        print(p_error)
        print("R: ")
        print(r_error)
        return p_error,r_error
    return True

## Display

### user input in scaner base (change it to robot world base)
### update Open3d in real time

In [84]:
def display_o3d(m1,m2,mesh=None,tn=None):
    p= m1.t.reshape((3,1))
    r= m1.R
    o_p = m2.t.reshape((3,1))
    o_r = m2.R
    marker = np.load('./test_data/marker2.npy')
    marker_3d_transformed = create_marker_visualization(marker, [0.2, 0.2, 0.8], o_r, o_p)
    marker_3d_transformed.append(create_coordinate_frame_visualization(o_r, o_p))
    marker = np.load('./test_data/marker1.npy')
    marker_3d_target = create_marker_visualization(marker, [0.8, 0.2, 0.8],r,p)
    marker_3d_target.append(create_coordinate_frame_visualization(r, p))
    if tn is not None:
        p= tn.t.reshape((3,1))
        r= tn.R
        marker_tn = create_marker_visualization(marker, [0.8, 0.2, 0.2],r,p)
        marker_tn.append(create_coordinate_frame_visualization(r, p))
    else:
        marker_tn = []
    if mesh is not None:
        o3d.visualization.draw_geometries(marker_3d_transformed + marker_3d_target + marker_tn + [mesh])
    else:
        o3d.visualization.draw_geometries(marker_3d_transformed + marker_3d_target + marker_tn)

In [102]:
def display_o3d_all(data):
    v_list = []
    for i in data:
        m1 = data[i][0].T
        m2 = data[i][1].T
        p= m1.t.reshape((3,1))
        r= m1.R
        o_p = m2.t.reshape((3,1))
        o_r = m2.R
        marker = np.load('./test_data/marker2.npy')
        marker_3d_transformed = create_marker_visualization(marker, [0.2, 0.2, 0.8], o_r, o_p)
        marker_3d_transformed.append(create_coordinate_frame_visualization(o_r, o_p))
        marker = np.load('./test_data/marker1.npy')
        marker_3d_target = create_marker_visualization(marker, [0.8, 0.2, 0.8],r,p)
        marker_3d_target.append(create_coordinate_frame_visualization(r, p))
        if i == 0:
            v_list = marker_3d_transformed + marker_3d_target
        else:
            v_list += marker_3d_transformed + marker_3d_target
    o3d.visualization.draw_geometries(v_list)

In [7]:
def check_det(a):
    print("The det of the Transfrom is: ", np.linalg.det(a[:3,:3]))
    print(np.linalg.det(a[:3,:3]) == 1)

## Pick DICOM files

In [123]:
data = {}
s_num = 0
cal_m1_list = []
fk_list = []
button = ['+x','-x','+y','-y','+z','-z','+r','-r','+θ','-θ','+φ','-φ']

In [137]:
data_directory = "./test_data/DICOM" #******* Change data_directory for using differnt scans ***********
# Global variable 'selected_series' is updated by the interact function
selected_series = ''
file_reader = sitk.ImageFileReader()
# Directory contains multiple DICOM studies/series, store
# in dictionary with key being the series ID
series_file_names = {}
series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(data_directory)
# Check that we have at least one series
if series_IDs:
    for series in series_IDs:
        series_file_names[series] = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, series)

    interact(DICOM_series_dropdown_callback, series_to_load=list(series_IDs), series_dictionary=fixed(series_file_names)); 
else:
    print('Data directory does not contain any DICOM series.')

interactive(children=(Dropdown(description='series_to_load', options=('1.2.840.113619.2.416.107981949962459061…

### Find End Efeector and Base Marker

In [138]:
time = datetime.datetime.now()
time = time.strftime("%m/%d/%y %I:%M%p")
marker1,marker2,mesh = get_marker(selected_series,time)

spacing:  [0.62 0.98 0.98]
image shape in sitk (x,y,z): width 512 then height 512 then depth 48
 origin:  [ -19.69 -250.   -250.  ]
numpy version shape: (z,y,x) (48, 512, 512)
locations of different pixels in image [(-250.0, -250.0, -19.687), (250.00025599999998, 250.00025599999998, 10.312999999999999)]
there math https://discourse.itk.org/t/solved-transformindextophysicalpoint-manually/1031/10
perhpas mcubes mirrors?
extreme simple itk pixels: [-992, -1006]
extreme simple itk pixels: [-992, -1006]
verts shape: (13358, 3)
verts type: <class 'numpy.ndarray'>
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=3
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [K

ValueError: zero-size array to reduction operation maximum which has no identity

### Display

In [92]:
display_o3d(marker1.T,marker2.T,mesh)

In [77]:
FK_robot_tip = np.array(([-1,0.1,0,-.3],[.01,1,.07,-1.81],[0,0.07,-1,1.18],[0,0,0,1]))

In [99]:
#Target = np.array(([8.21e-3,9.98e-1,5.45e-2,2.39e-2],[1.38e-1,-5.50e-2,9.89e-1,-1.75e-1],[9.90e-1,-6.8e-4,-1.38e-1,8.9e-2],[0,0,0,1]))
rbot_wrold_to_robot_base = np.array(([0,.04,-1,-0.21],[1,0.2,0,-1.87],[.02,-1,-.04,.98],[0,0,0,1]))
FK_robot_tip = trnorm(FK_robot_tip)
rbot_wrold_to_robot_base = trnorm(rbot_wrold_to_robot_base)
t2 = SE3(martix)
t2.t= t2.t*1000
t1 = SE3(FK_robot_tip)
t1.t= t1.t*1000

NameError: name 'FK_robot_tip' is not defined

In [59]:
cal_m1 = t2.inv()*t1
display_o3d(t1,t2)

  -0.007832  0.03919  -0.9992   -0.21      
   0.9806    0.196     0        -1.87      
   0.1958   -0.9798   -0.03997   0.98      
   0         0         0         1         

  -0.9951    0.09926   0        -0.3       
   0.09902   0.9926    0.06983  -1.81      
   0.006931  0.06948  -0.9976    1.18      
   0         0         0         1         

   0.1063    0.9862   -0.1269    0.0987    
  -0.02639   0.1303    0.9911   -0.1877    
   0.994    -0.102     0.03987   0.08193   
   0         0         0         1         



In [ ]:
cal_m1_list.append(cal_m1)
fk_list.append(FK_robot_tip)

In [140]:
append_value(data, "scan"+str(s_num), marker1)
append_value(data, "scan"+str(s_num), marker2)
d_name = 'crane_data_rotaion_test'+str(s_num)+'.pkl'
with open(d_name, 'wb') as f:
    pickle.dump(data, f)
s_num += 1

In [141]:
display_o3d_all(data)

In [ ]:
with open('./test_data/cal_m1.pkl', 'wb') as f:
    pickle.dump(cal_m1_list, f)

## Get Detail Mesh

In [16]:
convert_mha_to_mesh(output_mesh_file = 'temp_full_mesh.obj', threshold_value = 0)
temp_full_mesh = o3d.io.read_triangle_mesh('temp_full_mesh.obj').paint_uniform_color([0.7, 0.7, 0.7]).compute_vertex_normals()

spacing:  [0.62 0.98 0.98]
image shape in sitk (x,y,z): width 512 then height 512 then depth 256
 origin:  [-149.69 -250.   -250.  ]
numpy version shape: (z,y,x) (256, 512, 512)
locations of different pixels in image [(-250.0, -250.0, -149.687), (250.00025599999998, 250.00025599999998, 10.312999999999988)]
there math https://discourse.itk.org/t/solved-transformindextophysicalpoint-manually/1031/10
perhpas mcubes mirrors?
extreme simple itk pixels: [-195, -974]
extreme simple itk pixels: [-195, -974]
verts shape: (1559071, 3)
verts type: <class 'numpy.ndarray'>
[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2


# find Base maarker

In [ ]:
marker = np.load('./test_data/marker1.npy')
marker = marker - marker.mean(axis=0) - np.array([2,0,0])
crop_z = None
convert_scan_to_mha(series_file_names[selected_series], crop_z = crop_z)
convert_mha_to_mesh(output_mesh_file = 'temp_mesh.obj')
_, marker_centroid_coordinates, pcd_selected, marker_mesh, _, good_centroid_clusters = find_candidate_centroids(marker = marker, debug=False)
tracked_R, tracked_t, permuted_centroids, min_error = find_best_transform_from_candidate_marker_clusters(marker, good_centroid_clusters)
permuted_centroids = permuted_centroids[:3,:].T
transformed_marker = (tracked_R @ marker.T + tracked_t).T
marker2 = SE3(t2T(tracked_R, tracked_t))

In [ ]:
Target = np.array(([8.21e-3,9.98e-1,5.45e-2,2.39e-2],[1.38e-1,-5.50e-2,9.89e-1,-1.75e-1],[9.90e-1,-6.8e-4,-1.38e-1,8.9e-2],[0,0,0,1]))
martix = np.array(([0,.04,-1,-0.21],[1,0.2,0,-1.87],[.02,-1,-.04,.98],[0,0,0,1]))
Target = trnorm(Target)
martix = trnorm(martix)
check_det(martix)
check_det(Target)
rw_T = SE3(martix,check=False)*marker2.inv()*SE3(Target,check=False)
# Display in open3d
display_o3d(marker2,rw_T,temp_full_mesh)